In [1]:
# Adapted KERAS tutorial 
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, AlphaDropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization

/Users/JyotiSrivastava/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
import os
import pickle
import numpy as np

In [3]:
batch_size = 32
num_classes = 10
epochs = 20
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_batch_trained_model.h5'

In [4]:
# list devices so you can check whether your gpu is available
print(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [5]:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [6]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:],kernel_initializer='lecun_normal',bias_initializer='zeros'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3),kernel_initializer='lecun_normal',bias_initializer='zeros'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(AlphaDropout(0.1))

model.add(Conv2D(64, (3, 3), padding='same',kernel_initializer='lecun_normal',bias_initializer='zeros'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3),kernel_initializer='lecun_normal',bias_initializer='zeros'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(AlphaDropout(0.1))

model.add(Flatten())
model.add(Dense(512,kernel_initializer='lecun_normal',bias_initializer='zeros'))
model.add(BatchNormalization())
model.add(AlphaDropout(0.2))
model.add(Dense(num_classes,kernel_initializer='lecun_normal',bias_initializer='zeros'))
model.add(Activation('softmax'))

In [7]:
# initiate RMSprop optimizer
opt = keras.optimizers.legacy.RMSprop(learning_rate=0.0001)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [8]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [9]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test))

Using real-time data augmentation.
Epoch 1/20
1562/1562 [==============================] - 55s 35ms/step - loss: 1.8456 - accuracy: 0.3614 - val_loss: 2.5902 - val_accuracy: 0.4432
Epoch 2/20
1562/1562 [==============================] - 56s 36ms/step - loss: 1.5758 - accuracy: 0.4478 - val_loss: 1.9170 - val_accuracy: 0.5384
Epoch 3/20
1562/1562 [==============================] - 58s 37ms/step - loss: 1.4588 - accuracy: 0.4886 - val_loss: 1.9880 - val_accuracy: 0.5544
Epoch 4/20
1562/1562 [==============================] - 57s 36ms/step - loss: 1.3900 - accuracy: 0.5148 - val_loss: 1.7324 - val_accuracy: 0.5861
Epoch 5/20
1562/1562 [==============================] - 56s 36ms/step - loss: 1.3523 - accuracy: 0.5253 - val_loss: 1.6667 - val_accuracy: 0.5925
Epoch 6/20
1562/1562 [==============================] - 55s 35ms/step - loss: 1.3171 - accuracy: 0.5417 - val_loss: 1.5082 - val_accuracy: 0.6129
Epoch 7/20
1562/1562 [==============================] - 55s 35ms/step - loss: 1.2917 - ac

In [10]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at /Users/JyotiSrivastava/Documents/smai/SNNs/TF_2_x/saved_models/keras_cifar10_batch_trained_model.h5 


/Users/JyotiSrivastava/Library/Python/3.9/lib/python/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [12]:
# Load label names to use in prediction results
label_list_path = 'datasets/cifar-10-batches-py/batches.meta'


keras_dir = os.path.expanduser(os.path.join('~', '.keras'))
datadir_base = os.path.expanduser(keras_dir)
if not os.access(datadir_base, os.W_OK):
    datadir_base = os.path.join('/tmp', '.keras')
label_list_path = os.path.join(datadir_base, label_list_path)

with open(label_list_path, mode='rb') as f:
    labels = pickle.load(f)

# Evaluate model with test data set and share sample prediction results
evaluation = model.evaluate(datagen.flow(x_test, y_test,
                                      batch_size=batch_size),
                                      steps=x_test.shape[0] // batch_size)

print('Model Accuracy = %.5f' % (evaluation[1]))

f = open('CIFAR10_SELU_results.txt', 'a')
f.write(' Test accuracy:' + str(evaluation[1]) +  '\n')  
f.close() 


predict_gen = model.predict(datagen.flow(x_test, y_test,
                                      batch_size=batch_size),
                                      steps=x_test.shape[0] // batch_size)



312/312 [==============================] - 3s 10ms/step - loss: 1.3177 - accuracy: 0.6341
Model Accuracy = 0.63411
312/312 [==============================] - 3s 9ms/step
